<a href="https://colab.research.google.com/github/theharshithh/onfinace-assignment/blob/main/rerank_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
!pip -q install langchain openai tiktoken chromadb langchain_openai

In [ ]:
!pip install langchain_cohere cohere

In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain_cohere import CohereEmbeddings
import cohere
from google.colab import userdata
cohere_api_key = userdata.get('COHERE_API_KEY')

co = cohere.Client(cohere_api_key)


In [15]:
def read_doc(dir):
    file_loader = PyPDFDirectoryLoader(dir)
    docs = file_loader.load()
    return docs

loading data to /dataset dir

In [ ]:
!pip install pypdf

In [ ]:
primary_data = read_doc('dataset/')
primary_data

creating chunks for the `primary_data`

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
spilit_texts = text_splitter.split_documents(primary_data)
spilit_texts

## create the embeddings using hf ede by baai

In [20]:
def remove_discloses()

change of embeddings from `OpenAIEmbeddings()` to bge-embeddings

In [21]:
persist_directory = 'db-new'
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

#openai embeddings -> converts texts to vecs
embedding = OpenAIEmbeddings(openai_api_key = openai_api_key, model="text-embedding-3-small")

#load the spilt chunk vectors into the vectordb
vectordb = Chroma.from_documents(documents=spilit_texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [22]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [23]:
user_query = 'Considering 2 wheeler demand, who seems to be the segment winner?'

In [34]:
retriever = vectordb.as_retriever(search_kwargs={"k":8}) #k: 20 for more context window to llm
docs = retriever.get_relevant_documents(user_query)
docs #retriver gets the first layer of retrival docs

[Document(page_content='5% YoY , largely due to  the impact of  unseasonal rains and low yield s for key \ncrops  such as oni on. In Mar’23, overall wholesale volumes for \nPV/CV/tractor/3Ws are estimated to grow by 1%/ 1.5%/1 %/8% YoY, while 2Ws \nare likely to decline 4% YoY.  \n\uf06e 2Ws:  Retails are expected to grow 4-6% YoY in Mar ’23 since urban demand is \nsteady and inquiries have improved in certain states during the ongoing Navratri \nseason . However, overall sales  are still do wn by 15 -17% v /s the pre-Covid  level \nas rural demand remain s weak.  Enquiry levels during the key festival s (Gudi \nPadwa and Ugadi) varied greatly among  states.  We have noted 2-6% YoY growth \nin enquiry levels in states like Gujarat, Delhi and Karnataka , while  the same \ndeclined by 4-8% in other key states like UP, Bihar  and Maharashtra . The i nitial \nresponse to HMCL’ s new launch Xoom (110 cc scooter) has been positi ve as it has \naccounted for 25-30% of the overall enquiries in

### LLM chain

In [50]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

compressor = CohereRerank(cohere_api_key = cohere_api_key, model = 'rerank-english-v3.0') #   top_n = 3 -> can limit the sources
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever,
)

compressed_docs = compression_retriever.get_relevant_documents(user_query)
compressed_docs
# results = co.rerank(query=user_query, documents=docs, top_n=10, model='rerank-english-v3.0')

[Document(page_content="UV's  29,171  28,099  3.8 28,796  1.3 3,63,332  2,27,506  59.7  \nHero MotoCorp  4,34,165  4,50,170  -3.6 3,94,460  10.1  52,43,369  49,44,148  6.1 \nBajaj Auto  2,81,922  2,97,188  -5.1 2,80,226  0.6 39,16,077  43,08,433  -9.1 \n  Two -Wheelers  2,33,703  2,56,324  -8.8 2,35,356  -0.7 34,27,405  38,36,856  -10.7  \n  Three -Wheelers  48,219  40,864  18.0  44,870  7.5 4,88,672  4,71,577  3.6 \nDomestic  1,64,775  1,26,752  30.0  1,53,291  7.5 20,82,735  18,01,807  15.6  \nExports  1,17,146  1,70,436  -31.3  1,26,935  -7.7 18,33,341  25,06,626  -26.9  \nAshok Leyland  20,894  20,123  3.8 18,571  12.5  1,89,173  1,28,326  47.4  \nM&HCV  14,197  13,990  1.5 12,668  12.1  1,21,533  73,885  64.5  \nLCV  6,697  6,133  9.2 5,903  13.5  67,640  54,441  24 \nTVS Motor  2,92,177  3,07,954  -5.1 2,76,150  5.8 36,57,093  33,09,578  10.5  \nMotorcycles  1,37,970  1,60,522  -14.0  1,26,243  9.3 17,29,976  17,31,459  -0.1 \nScooters  1,05,785  94,747  11.6  1,04,825  0.9 12,97

In [52]:
# create the chain to answer question
qa_chain_compressed = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=1.2 ), chain_type="stuff", retriever=compression_retriever, return_source_documents=True)

In [48]:
# create the chain to answer question
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=1.2 ), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [40]:
def process_llm_response(llm_response):
    print("Decision:")
    print(llm_response['result'])

    print("\n Source Documents:")
    for source in llm_response["source_documents"]:
        print("- " + source.metadata['source'])

    print("\nRelevant Data:")
    for source in llm_response["source_documents"]:
        print(source.page_content)

compressed qa chain for efficent retrival

In [44]:
behaviour = '''Think like an expert finance consultant from a top tier firm. Do all the research and analysis for this document.'''
prompt_template = 'Keep the answer crisp. Give me as much numbers as you can. Every Statement of yours should be backed with data'
stop_prompt = f'Dont hallucinate, if u dont know the answer, tell you dont know'
query = f"Here is the user query : {user_query}. Remember this: {behaviour}. Give me the answers like this: {prompt_template}. Keep this in mind - {stop_prompt}"
llm_response = qa_chain_compressed(query)
process_llm_response(llm_response)

Decision:
Based on the provided data and analysis, the 2-wheeler segment winner based on demand seems to be TVS Motor. While the overall 2-wheeler segment experienced a decline in sales, TVS Motor showed resilience with a marginal 0.1% decrease in sales in FY23 compared to the previous year. This performance is notable compared to Hero MotoCorp, which saw a 6.1% decline in sales, and Bajaj Auto, which experienced a 9.1% decline.

Additionally, in terms of specific product performance within 2-wheelers, TVS Motor's scooter segment showed significant growth with an 11.6% increase in sales in Mar '23 compared to the previous year. This outperformed both the motorcycle segment, which saw a 14.0% decrease in sales, and the overall 2-wheeler segment, which experienced an 8.8% decline.

Therefore, based on the data provided, TVS Motor appears to be the segment winner in the 2-wheeler category, showcasing strong performance and potential for growth even in challenging market conditions.

 Sour

best version -> heavy credits burning -> deep pockets required

In [49]:
llm_response = qa_chain(query)
process_llm_response(llm_response)

Decision:
Based on the current market trends and research data, the segment winner in the 2-wheeler demand appears to be Hero MotoCorp. As per the report from March 2023, Hero MotoCorp is showing positive growth in two-wheeler sales volume. In comparison to other competitors like TVS Motor and Bajaj Auto, Hero MotoCorp exhibited a significant increase in sales volumes for motorcycles and scooters.

Here are some data points to support this analysis:
- Hero MotoCorp has a sales volume of 4,34,165 units, showcasing a positive growth trend compared to the previous period.
- Bajaj Auto reported sales of 2,81,922 units, which indicated a slight decline in sales.
- TVS Motor had sales of 2,92,177 units, showing a moderate decrease in sales volume.

Considering these numbers, Hero MotoCorp stands out as a strong contender in the 2-wheeler segment due to its positive sales growth compared to its competitors in the industry.

 Source Documents:
- dataset/sample_data.pdf
- dataset/sample_data.pd